<h1> EDA

<h3> Nesse jupyter notebook iremos analisar os dados de produtos e vendas da Vai Store

Primeiro vamos começar importando as bibliotecas que serão utilizadas ao longo do projeto

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

<h2> Produto

Vamos começar a nossa análise exploratória dos dados com o **produto**

In [ ]:
produtos = pd.read_csv('../data/raw/produto.csv', index_col = 0)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\felip\AppData\Local\Temp\ipykernel_2000\3475921097.py:1: SyntaxWarning: invalid escape sequence '\p'
  produtos = pd.read_csv('Dados Projeto\produto.csv', index_col = 0)


In [11]:
produtos.head()

,NOME_PRODUTO,CATEGORIA,SUBCATEGORIA
SKU,,,
3.0,COCO RALADO GROSSO KG ...,Doceria,Confeitaria
4.0,ICE TEA LEAO LATA 340ML ...,NaN,NaN
5.0,TAHINE ISTAMBUL 200G ...,Pelo Mundo,Pastas Árabes
6.0,AMENDOIM MOIDO KG ...,Castanhas & Oleaginosas,Oleaginosas moídas
7.0,HALAWI ISTAMBUL LATA 500G ...,Pelo Mundo,Pastas Árabes


Sumário das colunas:
* SKU = Código do produto 
* NOME_PRODUTO = Descrição do produto 
* CATEGORIA = Categoria do produto 
* SUBCATEGORIA = Subcategoria do produto

Vamos entender qual o número de diferentes produtos que a VAI Store vende

In [12]:
produtos.shape

(6500, 3)

São 6500 produtos diferentes vendidos. (Vale via código comprovar que não tem nenhum produto duplicado com o mesmo nome)

Podemos observar que na coluna identificada como **NOME_PRODUTO** temos o nome que o representa e também a unidade de massa ou volume em que ele é vendido. Podemos subdividir essa coluna em **NOME_PRODUTO** e **UNIDADE_VENDA** para depois poder trabalhar com esse dado de unidades.

<h2> Vendas

Agora vamos fazer a análise exploratória dos dados para as **vendas**

In [ ]:
vendas = pd.read_csv('../data/raw/vendas.csv', index_col = 0, )

In [17]:
vendas.head()

,COD_VENDA,COD_FILIAL,FILIAL,DATA_ATEND,SKU,UNID,QTD_VENDA,FATUR_VENDA,CLI_CPF
COD_ATEND,,,,,,,,,
000304564.299.0101042.20240102,000304564.299.0101042.20240102.000009.02,101042,SHOPPING,2024-01-02,9,KG,0.258,18.04,ec0abf3f4220
000374698.258.0101032.20240102,000374698.258.0101032.20240102.000009.10,101032,RUA,2024-01-02,9,KG,0.064,4.50,83d596ee0acf
000383551.256.0101032.20240102,000383551.256.0101032.20240102.000009.02,101032,RUA,2024-01-02,9,KG,0.164,11.49,46f63c18e055
000383533.256.0101032.20240102,000383533.256.0101032.20240102.000009.02,101032,RUA,2024-01-02,9,KG,0.202,14.13,6def1d35e2fe
000432095.257.0101032.20240102,000432095.257.0101032.20240102.000051.06,101032,RUA,2024-01-02,51,KG,0.086,6.87,1576e1a36394


Sumário das colunas:
* COD_ATEND = Código identificador do atendimento 
* COD_VENDA = Código identificador da venda 
* COD_FILIAL = Indica em qual loja foi feito o atendimento 
* FILIAL = Tipo de filial (Loja de Rua ou de Shopping) 
* DATA_ATEND = Data de atendimento daquele cliente a respeito daquele produto 
* SKU = Código do produto 
* UNID = Indica tipo do produto (granel ou prateleira) 
* QTD_VENDA = Quantidade vendida do produto 
* FATUR_VENDA  = Faturamento 
* CLI_CPF = Código do cliente

In [18]:
nan_count_CPF = vendas['CLI_CPF'].isna().sum()
print(f"Número de valores ausentes na coluna 'CLI_CPF': {nan_count_CPF}")

Número de valores ausentes na coluna 'CLI_CPF': 0
